<a href="https://colab.research.google.com/github/jeonjnh/python_stock_backtest/blob/main/Python_Finance_Monthly_Price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd
from pandas.tseries.offsets import BMonthBegin, BDay
from datetime import date

# from Chat GPT
def get_monthly_first_business_day_data(ticker, start_date, end_date):
    try:
        # 애플 주식 데이터 가져오기
        stock_data = yf.download(ticker, start=start_date, end=end_date)

        # 매월 첫 번째 영업일의 날짜 가져오기
        first_business_days = pd.date_range(start=start_date, end=end_date, freq=BMonthBegin())

        # 주식 가격 데이터에서 해당 날짜의 종가 가져오기
        monthly_close_prices = []
        for date in first_business_days:
            # 해당 날짜가 데이터에 있는지 확인
            if date in stock_data.index:
                monthly_close_prices.append(stock_data.loc[date]['Adj Close'])
            else:
                # 데이터에 없는 경우 다음 영업일로 이동
                next_business_day = date + BDay(1)
                while next_business_day not in stock_data.index:
                    next_business_day += BDay(1)
                monthly_close_prices.append(stock_data.loc[next_business_day]['Adj Close'])

        return pd.Series(monthly_close_prices, index=first_business_days)

    except Exception as e:
        print(f"Error fetching data: {e}")
        return pd.Series([])  # 빈 Series를 반환하여 예외 발생 시 빈 데이터를 처리

# 사용 예시
ticker = 'AAPL'
start_date = '2021-01-01'
end_date = '2023-12-31'
monthly_invest = 300

monthly_close_prices = get_monthly_first_business_day_data(ticker, start_date, end_date)
"""
if not monthly_close_prices.empty:
    print(monthly_close_prices)
else:
    print("No data available for the specified period.")
"""

[*********************100%%**********************]  1 of 1 completed


'\nif not monthly_close_prices.empty:\n    print(monthly_close_prices)\nelse:\n    print("No data available for the specified period.")\n'

In [2]:
df = pd.DataFrame({'date': monthly_close_prices.index, 'adj_close': monthly_close_prices.values})
df = df.set_index('date')
df['number'] = 300 / df['adj_close']
df['cum_num'] = df['number'].cumsum()
df.head()

,adj_close,number,cum_num
date,,,
2021-01-01,127.164185,2.359155,2.359155
2021-02-01,131.812073,2.275968,4.635122
2021-03-01,125.759926,2.385498,7.020620
2021-04-01,121.046005,2.478397,9.499017
2021-05-03,130.434448,2.300006,11.799022


In [3]:
total_invest = df['number'].count() * monthly_invest
total_invest

10800

In [4]:
today = str(date.today())
stock_data = yf.download(ticker, today)
total_price = df['number'].sum() * stock_data['Adj Close'][0]
total_price

[*********************100%%**********************]  1 of 1 completed


13049.45512316388

In [5]:
returns = total_price - total_invest
returns

2249.4551231638798